In [1]:
import os
import sys

os.chdir("../")

from src.exception import CustomException
from src.logger import logging


In [2]:
##entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: int
    params_is_augmentation: bool


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_file_path: Path

In [3]:
from src.utils.common import create_directories
from src.utils.common import read_yaml
from src.constants import constant_paths

CONFIG_FILE_PATH = constant_paths().get_paths()[0]
PARAMS_FILE_PATH = constant_paths().get_paths()[1]

[2023-07-28 16:55:07,288] 25 root INFO - Current working directory: c:\Users\pragn\Documents\MLOPS\final_CDC


In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        #create_directories([self.config.artifacts_dir])

    def prepare_callbacks(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        create_directories([config.root_dir])

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_file_path)

        create_directories([Path(model_checkpoint_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path= Path(config.checkpoint_model_file_path)
        )
        #returning a class with loaded data path files
        return prepare_callbacks_config
      
    def model_train_config(self) -> ModelTrainConfig:
        training = self.config.model_train
        prepare_base_model = self.config.base_model
        params = self.params
        training_data_path = os.path.join(self.config.data_ingestion.unzip_dir,"Dataset_CDC_category")
        create_directories([Path(training.root_dir)])

        model_train_config = ModelTrainConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path=self.config.base_model.updated_base_model_path,
            training_data=Path(training_data_path),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_is_augmentation= params.AUGMENTATION
        )

        return model_train_config

Configuration is done, now it is time to create Model training components.

In [5]:
from src.exception import CustomException
from src.logger import logging
from zipfile import ZipFile
import urllib.request as request
import time
import tensorflow as tf

In [6]:
class PrepareCallbacks:
    def __init__(self,config : PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)  

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_file_path),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [7]:
class Training:
    def __init__(self,config:ModelTrainConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):

        datageneratorkwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datageneratorkwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs)
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range= 0.2,
                **datageneratorkwargs)
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
                    directory = self.config.training_data,
                    subset = "training",
                    shuffle = True,
                    **dataflow_kwargs)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            steps_per_epoch = self.steps_per_epoch,
            epochs = self.config.params_epochs,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )

        self.save_model(path = self.config.trained_model_path, model = self.model)

In [8]:
## Pipeline

from src.exception import CustomException

try:
    configmanager = ConfigurationManager()
    prepare_callbacks_config = configmanager.prepare_callbacks()
    prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks()

    #training model
    model_train_config = configmanager.model_train_config()
    model_train = Training(model_train_config)
    model_train.get_base_model()
    model_train.train_valid_generator()
    model_train.train(callbacks_list)
    

except Exception as e:
    raise CustomException(e,sys)

[2023-07-28 16:55:07,371] 45 root INFO - yaml file: config\config.yaml loaded successfully
[2023-07-28 16:55:07,373] 45 root INFO - yaml file: params.yaml loaded successfully
[2023-07-28 16:55:07,374] 64 root INFO - created directory at: artifacts/prepare_callbacks
[2023-07-28 16:55:07,375] 64 root INFO - created directory at: artifacts\prepare_callbacks\checkpoint_model
[2023-07-28 16:55:07,376] 64 root INFO - created directory at: artifacts\prepare_callbacks\tensorboard_root_log
[2023-07-28 16:55:07,378] 64 root INFO - created directory at: artifacts\training
Found 1612 images belonging to 4 classes.
Found 6455 images belonging to 4 classes.
403/403 [==============================] - 502s 1s/step - loss: 8.7964 - accuracy: 0.5269 - val_loss: 3.4862 - val_accuracy: 0.7131


c:\Users\pragn\anaconda3\envs\final_CDC_env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    # If you want to use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Running on the GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found. Running on CPU.")


No GPU found. Running on CPU.
